In [ ]:
import torch

if torch.cuda.is_available():
    print('CUDA-enabled GPU found.')
else:
    print('No GPU found. Training will be carried out on CPU, which might be '
          'slower.\n\nIf running on Google Colab, you can request a GPU runtime by'
          ' clicking\n`Runtime/Change runtime type` in the top bar menu, then '
          'selecting \'GPU\'\nunder \'Hardware accelerator\'.')


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
!pip install mne
!pip install torch
!pip install matplotlib
!pip install scikit-learn
!pip install pandas

In [4]:
import os
import copy
import mne
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline



# enable GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:

# import mne
# from mne.datasets.sleep_physionet.age import fetch_data

# subjects, recordings = range(83), [1, 2]

# fnames = fetch_data(subjects=subjects, recording=recordings, on_missing='warn')
# mne.set_log_level('ERROR')  # To avoid flooding the cell outputs with messages

In [7]:
# fnames = [
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4001E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4001EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4002E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4002EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4011E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4011EH-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4012E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4012EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4021E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4021EH-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4022E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4022EJ-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4031E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4031EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4032E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4032EP-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4041E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4041EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4042E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4042EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4051E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4051EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4052E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4052EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4061E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4061EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4062E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4062EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4071E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4071EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4072E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4072EH-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4081E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4081EC-Hypnogram.edf'),
#     ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4082E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4082EP-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4091E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4091EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4092E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4092EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4101E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4101EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4102E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4102EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4111E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4111EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4112E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4112EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4121E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4121EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4122E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4122EV-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4131E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4131EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4141E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4141EU-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4142E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4142EU-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4151E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4151EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4152E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4152EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4161E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4161EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4162E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4162EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4171E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4171EU-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4172E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4172EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4181E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4181EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4182E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4182EC-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4191E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4191EP-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4192E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4192EV-Hypnogram.edf'),
# ('/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4201E0-PSG.edf', '/content/drive/MyDrive/physionet.org/files/sleep-edfx/1.0.0/sleep-cassette/SC4201EC-Hypnogram.edf')




# ]


In [8]:

fnames = [
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4001E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4001EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4002E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4002EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4011E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4011EH-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4012E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4012EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4021E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4021EH-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4022E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4022EJ-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4031E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4031EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4032E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4032EP-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4041E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4041EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4042E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4042EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4051E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4051EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4052E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4052EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4061E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4061EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4062E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4062EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4071E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4071EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4072E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4072EH-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4081E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4081EC-Hypnogram.edf'),
    ('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4082E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4082EP-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4091E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4091EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4092E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4092EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4101E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4101EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4102E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4102EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4111E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4111EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4112E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4112EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4121E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4121EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4122E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4122EV-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4131E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4131EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4141E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4141EU-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4142E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4142EU-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4151E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4151EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4152E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4152EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4161E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4161EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4162E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4162EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4171E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4171EU-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4172E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4172EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4181E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4181EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4182E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4182EC-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4191E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4191EP-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4192E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4192EV-Hypnogram.edf'),
('../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4201E0-PSG.edf', '../../data_set/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4201EC-Hypnogram.edf')

]


In [ ]:
fnames

In [10]:
def load_sleep_physionet_raw(raw_fname, annot_fname, load_eeg_only=True,
                             crop_wake_mins=30):

    mapping = {'EOG horizontal': 'eog',
               'Resp oro-nasal': 'misc',
               'EMG submental': 'misc',
               'Temp rectal': 'misc',
               'Event marker': 'misc'}
    exclude = mapping.keys() if load_eeg_only else ()

    raw = mne.io.read_raw_edf(raw_fname, exclude=exclude)
    annots = mne.read_annotations(annot_fname)
    raw.set_annotations(annots, emit_warning=False)
    if not load_eeg_only:
        raw.set_channel_types(mapping)

    if crop_wake_mins > 0:  # Cut start and end Wake periods
        # Find first and last sleep stages
        mask = [x[-1] in ['1', '2', '3', '4', 'R']
                for x in annots.description]
        sleep_event_inds = np.where(mask)[0]

        # Crop raw
        max_time = (raw.n_times - 1) / raw.info['sfreq']
        #print(max_time)
        tmin = annots[int(sleep_event_inds[0])]['onset'] - \
               crop_wake_mins * 60
        tmax = annots[int(sleep_event_inds[-1])]['onset'] + \
               crop_wake_mins * 60
        if tmin >= 0 and tmax <= max_time:   # data preprocessing removing tmax (%s) which are grater than or equal to the max
            raw.crop(tmin=tmin, tmax=tmax)
        else:
            pass

    # Rename EEG channels
    ch_names = {i: i.replace('EEG ', '')
                for i in raw.ch_names if 'EEG' in i}
    mne.rename_channels(raw.info, ch_names)

    # Save subject and recording information in raw.info
    basename = os.path.basename(raw_fname)
    subj_nb, rec_nb = int(basename[3:5]), int(basename[5])
    raw.info['subject_info'] = {'id': subj_nb, 'rec_id': rec_nb}

    return raw

In [11]:
# def load_sleep_physionet_raw(raw_fname, annot_fname, load_eeg_only=True,
#                              crop_wake_mins=30):

#     # Mapping and exclude variables
#     mapping = {'EOG horizontal': 'eog',
#                'Resp oro-nasal': 'misc',
#                'EMG submental': 'misc',
#                'Temp rectal': 'misc',
#                'Event marker': 'misc'}
#     exclude = mapping.keys() if load_eeg_only else ()
#     print("Mapping:", mapping)
#     print("Exclude Channels:", exclude)

#     # Load raw data
#     raw = mne.io.read_raw_edf(raw_fname, exclude=exclude)
#     annots = mne.read_annotations(annot_fname)
#     print("Annotations:", annots)
#     raw.set_annotations(annots, emit_warning=False)
    
#     if not load_eeg_only:
#         raw.set_channel_types(mapping)
#         print("Channel types set for non-EEG channels.")

#     if crop_wake_mins > 0:  # Cut start and end Wake periods
#         # Find first and last sleep stages
#         mask = [x[-1] in ['1', '2', '3', '4', 'R']
#                 for x in annots.description]
#         sleep_event_inds = np.where(mask)[0]
#         print("Sleep Event Indices:", sleep_event_inds)

#         # Crop raw data
#         max_time = (raw.n_times - 1) / raw.info['sfreq']
#         print("Max Time (seconds):", max_time)
#         tmin = annots[int(sleep_event_inds[0])]['onset'] - crop_wake_mins * 60
#         tmax = annots[int(sleep_event_inds[-1])]['onset'] + crop_wake_mins * 60
#         print("Crop Start (tmin):", tmin)
#         print("Crop End (tmax):", tmax)

#         if tmin >= 0 and tmax <= max_time:   
#             raw.crop(tmin=tmin, tmax=tmax)
#             print(f"Data cropped from {tmin} to {tmax} seconds.")
#         else:
#             print("No cropping applied due to invalid tmin or tmax.")

#     # Rename EEG channels
#     ch_names = {i: i.replace('EEG ', '')
#                 for i in raw.ch_names if 'EEG' in i}
#     print("Renamed Channel Names:", ch_names)
#     mne.rename_channels(raw.info, ch_names)

#     # Save subject and recording information in raw.info
#     basename = os.path.basename(raw_fname)
#     subj_nb, rec_nb = int(basename[3:5]), int(basename[5])
#     raw.info['subject_info'] = {'id': subj_nb, 'rec_id': rec_nb}
#     print("Subject Info:", raw.info['subject_info'])

#     return raw


In [ ]:
#data_loading

raws=[load_sleep_physionet_raw(f[0], f[1]) for f in fnames]

In [ ]:
# Plot a recording as a sanity check
raws[1].plot();
raws[0].plot();

In [ ]:
l_freq, h_freq = None, 30

for raw in raws:
    raw.load_data().filter(l_freq, h_freq)  # filtering happens in-place


# Plot the power spectrum of a recording as sanity check
raws[0].plot_psd();

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mne

# Load your raw data
# raws = [...]  # Assuming raws is a list of MNE Raw objects

l_freq, h_freq = None, 30

# Filter the data
for raw in raws:
    raw.load_data().filter(l_freq, h_freq)  # filtering happens in-place

# Plot the power spectrum of a recording as sanity check
fig, ax = plt.subplots(figsize=(10, 6))
raws[0].plot_psd(ax=ax, fmax=30, show=False)  # Plotting the PSD of the first raw data

# Define frequency bands and colors
bands = {
    'Delta (0-4 Hz)': (0, 4),
    'Theta (4-8 Hz)': (4, 8),
    'Low Alpha (8-10 Hz)': (8, 10),
    'High Alpha (10-12 Hz)': (10, 12),
    'Low Beta (12-18 Hz)': (12, 18),
    'High Beta (18-30 Hz)': (18, 30)
}

# Add shaded regions for frequency bands
for band, (low, high) in bands.items():
    ax.axvspan(low, high, color=np.random.rand(3,), alpha=0.3, label=band)  # Random color for each band

# Add labels and legend
ax.set_title('Power Spectrum with Frequency Bands')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power Spectral Density (dB/Hz)')
ax.legend(loc='upper right')

plt.show()


In [16]:
# function to extract 30-s windows

def extract_epochs(raw, chunk_duration=30.):

    annotation_desc_2_event_id = {
        'Sleep stage W': 1,
        'Sleep stage 1': 2,
        'Sleep stage 2': 3,
        'Sleep stage 3': 4,
        'Sleep stage 4': 4,
        'Sleep stage R': 5}

    events, _ = mne.events_from_annotations(
        raw, event_id=annotation_desc_2_event_id,
        chunk_duration=chunk_duration)

    # create a new event_id that unifies stages 3 and 4
    event_id = {
        'Sleep stage W': 1,
        'Sleep stage 1': 2,
        'Sleep stage 2': 3,
        'Sleep stage 3/4': 4,
        'Sleep stage R': 5}

    tmax = 30. - 1. / raw.info['sfreq']  # tmax in included
    picks = mne.pick_types(raw.info, eeg=True, eog=True)
    epochs = mne.Epochs(raw=raw, events=events, picks=picks, preload=True,
                        event_id=event_id, tmin=0., tmax=tmax, baseline=None)

    return epochs.get_data(), epochs.events[:, 2] - 1

In [17]:
# def extract_epochs(raw, chunk_duration=30.):

#     # Mapping of annotation descriptions to event IDs
#     annotation_desc_2_event_id = {
#         'Sleep stage W': 1,
#         'Sleep stage 1': 2,
#         'Sleep stage 2': 3,
#         'Sleep stage 3': 4,
#         'Sleep stage 4': 4,
#         'Sleep stage R': 5}
#     print("Annotation to Event ID Mapping:", annotation_desc_2_event_id)

#     # Extract events from annotations
#     events, _ = mne.events_from_annotations(
#         raw, event_id=annotation_desc_2_event_id,
#         chunk_duration=chunk_duration)
#     print("Events:", events)
    
#     # Create a unified event ID for stages 3 and 4
#     event_id = {
#         'Sleep stage W': 1,
#         'Sleep stage 1': 2,
#         'Sleep stage 2': 3,
#         'Sleep stage 3/4': 4,
#         'Sleep stage R': 5}
#     print("Unified Event ID Mapping:", event_id)

#     # Calculate tmax based on the raw data's sampling frequency
#     tmax = 30. - 1. / raw.info['sfreq']  # tmax is inclusive
#     print("tmax:", tmax)

#     # Pick channels (EEG and EOG)
#     picks = mne.pick_types(raw.info, eeg=True, eog=True)
#     print("Picked Channels:", picks)

#     # Create epochs
#     epochs = mne.Epochs(raw=raw, events=events, picks=picks, preload=True,
#                         event_id=event_id, tmin=0., tmax=tmax, baseline=None)
#     print("Epochs:", epochs)

#     # Return the epochs data and labels (adjust labels by subtracting 1)
#     data = epochs.get_data()
#     labels = epochs.events[:, 2] - 1
#     print("Epoch Data Shape:", data.shape)
#     print("Epoch Labels:", labels)

#     return data, labels


In [18]:
# wrap arounfd around pytorch dataset class
from torch.utils.data import Dataset, ConcatDataset

class EpochsDataset(Dataset):

    def __init__(self,epochs_data, epochs_labels, subj_nb=None, rec_nb=None, transform =None):
        assert len(epochs_data) == len(epochs_labels)
        self.epochs_data =epochs_data
        self.epochs_labels = epochs_labels
        self.subj_nb = subj_nb
        self.rec_nb = rec_nb
        self.transform = transform
    def __len__(self):
        return len(self.epochs_labels)
    def __getitem__(self,idx):
        X, y = self.epochs_data[idx], self.epochs_labels[idx]
        if self.transform is not None:
            X = self.transform(X)
        X = torch.as_tensor(X[None, ...])
        return X, y

#Standard scaling of data

def scale(X):

    X -= np.mean(X, axis=1, keepdims=True)
    return X / np.std(X, axis=1, keepdims=True)

In [19]:
import torch
import numpy as np
from torch.utils.data import Dataset

class EpochsDataset(Dataset):

    def __init__(self, epochs_data, epochs_labels, subj_nb=None, rec_nb=None, transform=None):
        # Ensure the lengths of data and labels match
        assert len(epochs_data) == len(epochs_labels)
        self.epochs_data = epochs_data
        self.epochs_labels = epochs_labels
        self.subj_nb = subj_nb
        self.rec_nb = rec_nb
        self.transform = transform
        # Print all the initialization variables
        print("Initialized EpochsDataset")
        print("Epochs Data Shape:", epochs_data.shape)
        print("Epochs Labels Shape:", epochs_labels.shape)
        print("Subject Number:", subj_nb)
        print("Recording Number:", rec_nb)
        print("Transform Function:", transform)

    def __len__(self):
        # Print the length of the dataset when called
        length = len(self.epochs_labels)
        print("Dataset Length:", length)
        return length

    def __getitem__(self, idx):
        # Print the index being accessed
        print("Fetching data for index:", idx)
        X, y = self.epochs_data[idx], self.epochs_labels[idx]
        print("Raw Data (X):", X)
        print("Label (y):", y)

        # Apply transform if provided
        if self.transform is not None:
            print("Applying transform to data")
            X = self.transform(X)
        else:
            print("No transform applied")
        
        # Add a new axis and convert to tensor
        X = torch.as_tensor(X[None, ...])
        print("Transformed Data (X) Tensor Shape:", X.shape)

        return X, y

# Standard scaling of data
def scale(X):
    print("Original Data (X):", X)
    
    # Calculate the mean and standard deviation
    mean_X = np.mean(X, axis=1, keepdims=True)
    std_X = np.std(X, axis=1, keepdims=True)
    
    print("Mean of X:", mean_X)
    print("Standard Deviation of X:", std_X)

    # Perform standard scaling
    X -= mean_X
    scaled_X = X / std_X
    print("Scaled Data (X):", scaled_X)
    
    return scaled_X


In [ ]:

raws = [load_sleep_physionet_raw(f[0], f[1]) for f in fnames]

# If you want to test with only two datasets
raws_exc = [raws[1]]  # Remove the dataset you want to exclude
all_datasets = [EpochsDataset(*extract_epochs(raw), subj_nb=raw.info['subject_info']['id'], rec_nb=raw.info['subject_info']['rec_id'], transform=scale)
                for i, raw in enumerate(raws) if i not in raws_exc]

# Concatenate into a single dataset
dataset = ConcatDataset(all_datasets)


In [21]:
from sklearn.model_selection import LeavePGroupsOut


def pick_recordings(dataset, subj_rec_nbs):

    pick_idx = []
    for subj_nb, rec_nb in subj_rec_nbs:
        for i, ds in enumerate(dataset.datasets):
            if (ds.subj_nb == subj_nb) and (ds.rec_nb == rec_nb):
                pick_idx.append(i)

    remaining_idx = np.setdiff1d(
        range(len(dataset.datasets)), pick_idx)

    pick_ds = ConcatDataset([dataset.datasets[i] for i in pick_idx])
    if len(remaining_idx) > 0:
        remaining_ds = ConcatDataset(
            [dataset.datasets[i] for i in remaining_idx])
    else:
        remaining_ds = None

    return pick_ds, remaining_ds


def train_test_split(dataset, n_groups, split_by='subj_nb'):

    groups = [getattr(ds, split_by) for ds in dataset.datasets]
    train_idx, test_idx = next(
        LeavePGroupsOut(n_groups).split(X=groups, groups=groups))

    train_ds = ConcatDataset([dataset.datasets[i] for i in train_idx])
    test_ds = ConcatDataset([dataset.datasets[i] for i in test_idx])

    return train_ds, test_ds

In [22]:
# from sklearn.model_selection import LeavePGroupsOut
# import numpy as np
# from torch.utils.data import ConcatDataset

# def pick_recordings(dataset, subj_rec_nbs):

#     print(f"Input subject and recording numbers: {subj_rec_nbs}")
#     pick_idx = []
    
#     # Iterate over all subject and recording numbers
#     for subj_nb, rec_nb in subj_rec_nbs:
#         print(f"Checking for Subject: {subj_nb}, Recording: {rec_nb}")
#         for i, ds in enumerate(dataset.datasets):
#             if (ds.subj_nb == subj_nb) and (ds.rec_nb == rec_nb):
#                 pick_idx.append(i)
#                 print(f"Match found at index {i}")
    
#     print(f"Picked indices: {pick_idx}")

#     # Find the remaining indices
#     remaining_idx = np.setdiff1d(range(len(dataset.datasets)), pick_idx)
#     print(f"Remaining indices: {remaining_idx}")

#     # Create picked dataset and remaining dataset
#     pick_ds = ConcatDataset([dataset.datasets[i] for i in pick_idx])
#     print(f"Picked Dataset: {pick_ds}")

#     if len(remaining_idx) > 0:
#         remaining_ds = ConcatDataset([dataset.datasets[i] for i in remaining_idx])
#         print(f"Remaining Dataset: {remaining_ds}")
#     else:
#         remaining_ds = None
#         print("No remaining datasets.")

#     return pick_ds, remaining_ds

# def train_test_split(dataset, n_groups, split_by='subj_nb'):

#     print(f"Splitting dataset by: {split_by} with {n_groups} groups")
    
#     # Extract groups based on the split_by attribute
#     groups = [getattr(ds, split_by) for ds in dataset.datasets]
#     print(f"Groups: {groups}")

#     # Perform LeavePGroupsOut split
#     splitter = LeavePGroupsOut(n_groups)
#     train_idx, test_idx = next(splitter.split(X=groups, groups=groups))
    
#     print(f"Train indices: {train_idx}")
#     print(f"Test indices: {test_idx}")

#     # Create train and test datasets
#     train_ds = ConcatDataset([dataset.datasets[i] for i in train_idx])
#     test_ds = ConcatDataset([dataset.datasets[i] for i in test_idx])

#     print(f"Train Dataset: {train_ds}")
#     print(f"Test Dataset: {test_ds}")

#     return train_ds, test_ds


In [23]:
# We seed the random number generators to make our splits reproducible
torch.manual_seed(103)
np.random.seed(103)

In [ ]:
# Use recording 1 of subjects 0-19 as test set
test_recs = [(subj_nb, rec_nb)  #
             for subj_nb, rec_nb in zip(range(20), [1] * 10)]
test_ds, train_ds = pick_recordings(dataset, test_recs)

In [ ]:
# Split remaining recordings into training and validation sets
n_subjects_valid = max(1, int(len(train_ds.datasets) * 0.2))
train_ds, valid_ds = train_test_split(train_ds, n_subjects_valid, split_by='subj_nb')

print('Number of examples in each set:')
print(f'Training: {len(train_ds)}')
print(f'Validation: {len(valid_ds)}')
print(f'Test: {len(test_ds)}')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Data for each set
datasets = ['Training', 'Validation', 'Test']
examples_count = [len(train_ds), len(valid_ds), len(test_ds)]

# Create a custom colormap from blue to yellow
colors = sns.color_palette("coolwarm", as_cmap=False, n_colors=len(datasets))

# Create the bar plot
plt.figure(figsize=(8, 5))
sns.barplot(x=datasets, y=examples_count, palette=colors)

# Add labels and title
plt.xlabel('Dataset')
plt.ylabel('Number of Examples')
plt.title('Number of Examples in Each Dataset')

# Show the plot
plt.show()


In [ ]:
classes_mapping = {0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}
y_train = pd.Series([y for _, y in train_ds]).map(classes_mapping)
ax = y_train.value_counts().plot(kind='barh')
ax.set_xlabel('Number of training examples');
ax.set_ylabel('Sleep stage');

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Example data and mapping (assuming train_ds is already defined)
classes_mapping = {0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}
y_train = pd.Series([y for _, y in train_ds]).map(classes_mapping)

# Create the horizontal bar plot
fig, ax = plt.subplots(figsize=(10, 6))
value_counts = y_train.value_counts()

# Define a color palette (one color for each bar)
colors = ['mediumseagreen', 'lightcoral', 'steelblue', 'goldenrod', 'orchid']

# Plot with different colors for each bar
bars = ax.barh(value_counts.index, value_counts.values, color=colors, edgecolor='black', hatch='')

# Set labels and title with custom styling
ax.set_xlabel('Number of Training Examples', fontsize=12, color='black')
ax.set_ylabel('Sleep Stage', fontsize=12, color='black')
ax.set_title('Distribution of Training Examples by Sleep Stage', fontsize=14, color='black')

# Customize the plot background and grid
ax.set_facecolor('lightgray')
ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')

# Add bar labels with numeric values outside the bars
for bar in bars:
    width = bar.get_width()
    ax.annotate(f'{width}',
                xy=(width, bar.get_y() + bar.get_height() / 2),
                xytext=(5, 0),  # 5 points horizontal offset to avoid touching the rectangle
                textcoords="offset points",
                ha='left', va='center', fontsize=10, color='black')

# Add a legend without the rectangle (only the name "Training Examples")
# Create a blank Rectangle and add it to the legend
blank_patch = Rectangle((0, 0), 1, 1, fill=False, edgecolor='none', visible=False)
ax.legend([blank_patch], ['Training Examples'], loc='upper right', fontsize=10)

# Adjust layout for better appearance
plt.tight_layout()
plt.show()


In [ ]:
# Computing class weight
from sklearn.utils.class_weight import compute_class_weight

train_y = np.concatenate([ds.epochs_labels for ds in train_ds.datasets])
class_weights = compute_class_weight('balanced', classes=np.unique(train_y), y=train_y)
print(class_weights)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the class names and their corresponding weights
class_names = ['N3', 'N1', 'R', 'W', 'N2']
# class_weights = np.array([0.2, 0.3, 0.1, 0.1, 0.3])  # Replace with your actual class weights
class_weights = [0.95528734, 3.6621547,  0.47941416, 1.57213163, 1.04365282]
# Define colors for the pie chart
colors = plt.cm.Paired.colors[:len(class_names)]

# Plotting the pie chart with class weights
plt.figure(figsize=(8, 8))
patches, texts, autotexts = plt.pie(class_weights, labels=class_names, autopct='%1.1f%%', colors=colors, startangle=90)

# Adding title
plt.title('Class Weights Distribution (Percentage)', fontsize=14)

# Adding legend
plt.legend(patches, [f'Class {cls}' for cls in class_names], loc="best")

# Beautifying the chart
for text in autotexts:
    text.set_color('white')  # Make the percentage text color white for better readability

# Show the pie chart
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()


In [62]:
# echo "# task" >> README.md
# git init
# git add README.md
# git commit -m "first commit"
# git branch -M main
# git remote add origin https://github.com/mitaliptl-005/task.git
# # git push -u origin main

In [31]:
from torch import nn


class SleepStagerChambon2018(nn.Module):

    def __init__(self, n_channels, sfreq, n_conv_chs=8, time_conv_size_s=0.5,
                 max_pool_size_s=0.125, n_classes=5, input_size_s=30,
                 dropout=0.25):
        super().__init__()

        time_conv_size = int(time_conv_size_s * sfreq)
        max_pool_size = int(max_pool_size_s * sfreq)
        input_size = int(input_size_s * sfreq)
        pad_size = time_conv_size // 2
        self.n_channels = n_channels
        len_last_layer = self._len_last_layer(
            n_channels, input_size, max_pool_size, n_conv_chs)

        if n_channels > 1:
            self.spatial_conv = nn.Conv2d(1, n_channels, (n_channels, 1))

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(
                1, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size)),
            nn.Conv2d(
                n_conv_chs, n_conv_chs, (1, time_conv_size),
                padding=(0, pad_size)),
            nn.ReLU(),
            nn.MaxPool2d((1, max_pool_size))
        )
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(len_last_layer, n_classes)
        )

    @staticmethod
    def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
        return n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs

    def forward(self, x):

        if self.n_channels > 1:
            x = self.spatial_conv(x)
            x = x.transpose(1, 2)

        x = self.feature_extractor(x)
        return self.fc(x.flatten(start_dim=1))

In [32]:
# from torch import nn
# import torch

# class SleepStagerChambon2018(nn.Module):

#     def __init__(self, n_channels, sfreq, n_conv_chs=8, time_conv_size_s=0.5,
#                  max_pool_size_s=0.125, n_classes=5, input_size_s=30,
#                  dropout=0.25):
#         super().__init__()

#         time_conv_size = int(time_conv_size_s * sfreq)
#         max_pool_size = int(max_pool_size_s * sfreq)
#         input_size = int(input_size_s * sfreq)
#         pad_size = time_conv_size // 2
#         self.n_channels = n_channels
#         print(f"Initialization - n_channels: {n_channels}, sfreq: {sfreq}, "
#               f"time_conv_size: {time_conv_size}, max_pool_size: {max_pool_size}, "
#               f"input_size: {input_size}, pad_size: {pad_size}")

#         len_last_layer = self._len_last_layer(
#             n_channels, input_size, max_pool_size, n_conv_chs)
#         print(f"Length of the last layer: {len_last_layer}")

#         if n_channels > 1:
#             self.spatial_conv = nn.Conv2d(1, n_channels, (n_channels, 1))
#             print(f"Spatial Conv Layer created: {self.spatial_conv}")

#         self.feature_extractor = nn.Sequential(
#             nn.Conv2d(1, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
#             nn.ReLU(),
#             nn.MaxPool2d((1, max_pool_size)),
#             nn.Conv2d(n_conv_chs, n_conv_chs, (1, time_conv_size), padding=(0, pad_size)),
#             nn.ReLU(),
#             nn.MaxPool2d((1, max_pool_size))
#         )
#         print(f"Feature extractor: {self.feature_extractor}")

#         self.fc = nn.Sequential(
#             nn.Dropout(dropout),
#             nn.Linear(len_last_layer, n_classes)
#         )
#         print(f"Fully connected layer: {self.fc}")

#     @staticmethod
#     def _len_last_layer(n_channels, input_size, max_pool_size, n_conv_chs):
#         len_last_layer = n_channels * (input_size // (max_pool_size ** 2)) * n_conv_chs
#         print(f"Computed length of last layer - n_channels: {n_channels}, input_size: {input_size}, "
#               f"max_pool_size: {max_pool_size}, n_conv_chs: {n_conv_chs} -> len_last_layer: {len_last_layer}")
#         return len_last_layer

#     def forward(self, x):
#         print(f"Forward pass input: {x.shape}")
        
#         if self.n_channels > 1:
#             x = self.spatial_conv(x)
#             print(f"After spatial conv: {x.shape}")
#             x = x.transpose(1, 2)
#             print(f"After transpose: {x.shape}")

#         x = self.feature_extractor(x)
#         print(f"After feature extraction: {x.shape}")

#         x = x.flatten(start_dim=1)
#         print(f"After flattening: {x.shape}")

#         x = self.fc(x)
#         print(f"After fully connected layer: {x.shape}")
        
#         return x


In [33]:
# hyperperameters
dropout=0.75
lr=1e-3

In [34]:
sfreq = raws[0].info['sfreq']  # Sampling frequency
n_channels = raws[0].info['nchan']  # Number of channels

model = SleepStagerChambon2018(n_channels, sfreq, n_classes=5,dropout=dropout)

In [35]:
model = model.to(device)

In [36]:
from torch.utils.data import DataLoader

# Create dataloaders
train_batch_size = 128  # Important hyperparameter
valid_batch_size = 250  # Can be made as large as what fits in memory; won't impact performance
num_workers = 0  # Number of processes to use for the data loading process; 0 is the main Python process

loader_train = DataLoader(
    train_ds, batch_size=train_batch_size, shuffle=True, num_workers=num_workers)
loader_valid = DataLoader(
    valid_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)
loader_test = DataLoader(
    test_ds, batch_size=valid_batch_size, shuffle=False, num_workers=num_workers)

In [37]:
# functions to carry out our training and validation loops:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score

def _do_train(model, loader, optimizer, criterion, device, metric):
    # training loop
    model.train()

    train_loss = np.zeros(len(loader))
    y_pred_all, y_true_all = [], []
    for idx_batch, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()
        batch_x = batch_x.to(device=device, dtype=torch.float32)
        batch_y = batch_y.to(device=device, dtype=torch.int64)

        output = model(batch_x)
        loss = criterion(output, batch_y)

        loss.backward()
        optimizer.step()

        y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
        y_true_all.append(batch_y.cpu().numpy())

        train_loss[idx_batch] = loss.item()

    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    perf = metric(y_true, y_pred)

    return np.mean(train_loss), perf


def _validate(model, loader, criterion, device, metric):
    # validation loop
    model.eval()

    val_loss = np.zeros(len(loader))
    y_pred_all, y_true_all = [], []
    with torch.no_grad():
        for idx_batch, (batch_x, batch_y) in enumerate(loader):
            batch_x = batch_x.to(device=device, dtype=torch.float32)
            batch_y = batch_y.to(device=device, dtype=torch.int64)
            output = model.forward(batch_x)

            loss = criterion(output, batch_y)
            val_loss[idx_batch] = loss.item()

            y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
            y_true_all.append(batch_y.cpu().numpy())

    y_pred = np.concatenate(y_pred_all)
    y_true = np.concatenate(y_true_all)
    perf = metric(y_true, y_pred)

    return np.mean(val_loss), perf


def train(model, loader_train, loader_valid, optimizer, criterion, n_epochs,
          patience, device, metric=None):

    best_valid_loss = np.inf
    best_model = copy.deepcopy(model)
    waiting = 0
    history = []

    if metric is None:
        metric = balanced_accuracy_score

    print('epoch \t train_loss \t valid_loss \t train_perf \t valid_perf')
    print('-------------------------------------------------------------------')

    for epoch in range(1, n_epochs + 1):
        train_loss, train_perf = _do_train(
            model, loader_train, optimizer, criterion, device, metric=metric)
        valid_loss, valid_perf = _validate(
            model, loader_valid, criterion, device, metric=metric)
        history.append(
            {'epoch': epoch,
             'train_loss': train_loss, 'valid_loss': valid_loss,
             'train_perf': train_perf, 'valid_perf': valid_perf})

        print(f'{epoch} \t {train_loss:0.4f} \t {valid_loss:0.4f} '
              f'\t {train_perf:0.4f} \t {valid_perf:0.4f}')

        # model saving
        if valid_loss < best_valid_loss:
            print(f'best val loss {best_valid_loss:.4f} -> {valid_loss:.4f}')
            best_valid_loss = valid_loss
            best_model = copy.deepcopy(model)
            waiting = 0
        else:
            waiting += 1

        # model early stopping
        if waiting >= patience:
            print(f'Stop training at epoch {epoch}')
            print(f'Best val loss : {best_valid_loss:.4f}')
            break

    return best_model, history

In [38]:
# import numpy as np
# import copy
# from sklearn.metrics import balanced_accuracy_score

# def _do_train(model, loader, optimizer, criterion, device, metric):
#     # training loop
#     model.train()
    
#     train_loss = np.zeros(len(loader))
#     y_pred_all, y_true_all = [], []
    
#     for idx_batch, (batch_x, batch_y) in enumerate(loader):
#         optimizer.zero_grad()
        
#         # Move to device
#         batch_x = batch_x.to(device=device, dtype=torch.float32)
#         batch_y = batch_y.to(device=device, dtype=torch.int64)

#         output = model(batch_x)
#         loss = criterion(output, batch_y)

#         # Backpropagation
#         loss.backward()
#         optimizer.step()

#         y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
#         y_true_all.append(batch_y.cpu().numpy())

#         train_loss[idx_batch] = loss.item()

#         # Print variables for debugging
#         print(f"Batch {idx_batch}: Loss: {loss.item()}, Predictions: {y_pred_all[-1]}, True labels: {y_true_all[-1]}")

#     y_pred = np.concatenate(y_pred_all)
#     y_true = np.concatenate(y_true_all)
#     perf = metric(y_true, y_pred)

#     # Print overall performance metrics
#     print(f"Training - Loss: {np.mean(train_loss)}, Performance: {perf}")

#     return np.mean(train_loss), perf


# def _validate(model, loader, criterion, device, metric):
#     # validation loop
#     model.eval()

#     val_loss = np.zeros(len(loader))
#     y_pred_all, y_true_all = [], []
    
#     with torch.no_grad():
#         for idx_batch, (batch_x, batch_y) in enumerate(loader):
#             batch_x = batch_x.to(device=device, dtype=torch.float32)
#             batch_y = batch_y.to(device=device, dtype=torch.int64)
#             output = model(batch_x)

#             loss = criterion(output, batch_y)
#             val_loss[idx_batch] = loss.item()

#             y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
#             y_true_all.append(batch_y.cpu().numpy())

#             # Print variables for debugging
#             print(f"Validation Batch {idx_batch}: Loss: {loss.item()}, Predictions: {y_pred_all[-1]}, True labels: {y_true_all[-1]}")

#     y_pred = np.concatenate(y_pred_all)
#     y_true = np.concatenate(y_true_all)
#     perf = metric(y_true, y_pred)

#     # Print overall performance metrics
#     print(f"Validation - Loss: {np.mean(val_loss)}, Performance: {perf}")

#     return np.mean(val_loss), perf


# def train(model, loader_train, loader_valid, optimizer, criterion, n_epochs,
#           patience, device, metric=None):

#     best_valid_loss = np.inf
#     best_model = copy.deepcopy(model)
#     waiting = 0
#     history = []

#     if metric is None:
#         metric = balanced_accuracy_score

#     print('epoch \t train_loss \t valid_loss \t train_perf \t valid_perf')
#     print('-------------------------------------------------------------------')

#     for epoch in range(1, n_epochs + 1):
#         train_loss, train_perf = _do_train(
#             model, loader_train, optimizer, criterion, device, metric=metric)
#         valid_loss, valid_perf = _validate(
#             model, loader_valid, criterion, device, metric=metric)
        
#         history.append(
#             {'epoch': epoch,
#              'train_loss': train_loss, 'valid_loss': valid_loss,
#              'train_perf': train_perf, 'valid_perf': valid_perf})

#         print(f'{epoch} \t {train_loss:0.4f} \t {valid_loss:0.4f} '
#               f'\t {train_perf:0.4f} \t {valid_perf:0.4f}')

#         # model saving
#         if valid_loss < best_valid_loss:
#             print(f'best val loss {best_valid_loss:.4f} -> {valid_loss:.4f}')
#             best_valid_loss = valid_loss
#             best_model = copy.deepcopy(model)
#             waiting = 0
#         else:
#             waiting += 1

#         # model early stopping
#         if waiting >= patience:
#             print(f'Stop training at epoch {epoch}')
#             print(f'Best val loss : {best_valid_loss:.4f}')
#             break

#     return best_model, history


In [39]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=lr, weight_decay=0)
criterion = CrossEntropyLoss(weight=torch.Tensor(class_weights).to(device))

In [ ]:
print(optimizer)

In [ ]:
n_epochs =10
patience =10
best_model, history = train(model, loader_train, loader_valid, optimizer, criterion, n_epochs, patience,
                                 device, metric=cohen_kappa_score)

In [ ]:
best_epoch_index = max(range(len(history)), key=lambda i: history[i]['valid_perf'])
best_epoch_info = history[best_epoch_index]
best_epoch_performance = best_epoch_info['valid_perf']

# Print the results
print(f"Best Epoch Index: {best_epoch_index}")
print(f"Best Validation Performance: {best_epoch_performance}")

In [ ]:
# Compute test performance

best_model.eval()

y_pred_all, y_true_all = [], []
for batch_x, batch_y in loader_test:
    batch_x = batch_x.to(device=device, dtype=torch.float32)
    batch_y = batch_y.to(device=device, dtype=torch.int64)
    output = model.forward(batch_x)
    y_pred_all.append(torch.argmax(output, axis=1).cpu().numpy())
    y_true_all.append(batch_y.cpu().numpy())

y_pred = np.concatenate(y_pred_all)
y_true = np.concatenate(y_true_all)
rec_ids = np.concatenate(  # indicates which recording each example comes from
    [[i] * len(ds) for i, ds in enumerate(test_ds.datasets)])

test_bal_acc = balanced_accuracy_score(y_true, y_pred)
test_kappa = cohen_kappa_score(y_true, y_pred)

print(f'Test balanced accuracy: {test_bal_acc:0.3f}')
print(f'Test Cohen\'s kappa: {test_kappa:0.3f}')

In [141]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(conf_mat, classes_mapping):
    ticks = [key for key in classes_mapping.keys()]
    tick_labels = classes_mapping.values()

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_mat, cmap='Greens')

    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    ax.set_title('Confusion matrix')

    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(
                j, i, conf_mat[i, j], ha='center', va='center', color='k')

    fig.colorbar(im, ax=ax, fraction=0.05, label='# examples')
    fig.tight_layout()

    return fig, ax

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define metrics and their names
metrics = ['Balanced Accuracy', 'Cohen\'s Kappa']
values = [test_bal_acc, test_kappa]
errors = [0.02, 0.02]  # Example error values; replace with actual if available

# Create the bar plot
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(metrics, values, yerr=errors, capsize=5, color=['skyblue', 'lightcoral'])

# Add labels and title
ax.set_xlabel('Metric')
ax.set_ylabel('Score')
ax.set_title('Test Performance Metrics Comparison')
ax.set_ylim(0, 1)  # Set y-axis limit from 0 to 1 for percentage metrics

# Adding value labels on top of bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.02, f'{yval:.3f}', ha='center', va='bottom')

plt.show()


In [ ]:
conf_mat = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(conf_mat, classes_mapping);

In [144]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(conf_mat, classes_mapping):
    ticks = list(classes_mapping.keys())
    tick_labels = classes_mapping.values()

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_mat, cmap='Reds')

    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    ax.set_title('Confusion matrix')

    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(
                j, i, conf_mat[i, j], ha='center', va='center', color='k')

    fig.colorbar(im, ax=ax, fraction=0.05, label='')
    fig.tight_layout()

    return fig, ax

In [ ]:
conf_mat = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(conf_mat, classes_mapping);

In [ ]:
import seaborn as sns
x_axis_labels=["W", "N1", 'N2','N3','R']
y_axis_labels =["W", 'N1', 'N2','N3','R']

ax = sns.heatmap(conf_mat/conf_mat.sum(axis=1)[:, np.newaxis], annot=True,  cmap='Greens',xticklabels=x_axis_labels, yticklabels=y_axis_labels)
ax.set_yticklabels(ax.get_yticklabels(), rotation = 0, fontsize = 10)
ax.set_title('Confusion Matrix with precentage \n\n');
ax.set_xlabel('\nPredicted label')
ax.set_ylabel('True label');


## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
from sklearn import metrics
target_names = ['W', 'N1', 'N2','N3','R']
print(metrics.classification_report(y_true, y_pred, target_names=target_names))

In [148]:
y_pred = np.concatenate(y_pred_all)
y_true = np.concatenate(y_true_all)


In [ ]:
import torch.nn.functional as F

def evaluate_model(model, loader, device):
    model.eval()  # Set model to evaluation mode
    true_labels = []
    predictions = []
    with torch.no_grad():  # No need to track gradients
        for batch_x, batch_y in loader:
            batch_x = batch_x.to(device=device, dtype=torch.float32)
            output = model(batch_x)
            probs = F.softmax(output, dim=1)  # Convert logits to probabilities
            predictions.append(probs.cpu().numpy())
            true_labels.append(batch_y.cpu().numpy())

    predictions = np.concatenate(predictions)
    true_labels = np.concatenate(true_labels)
    return true_labels, predictions

# After training your model, evaluate it to get true labels and predicted probabilities
y_true, y_pred_probs = evaluate_model(best_model, loader_test, device)


In [ ]:
from sklearn.metrics import roc_curve, auc
from itertools import cycle
import matplotlib.pyplot as plt

def plot_multiclass_roc_auc(y_true, y_score, n_classes):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true == i, y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot ROC curve for each class
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Call the plotting function with the number of classes
plot_multiclass_roc_auc(y_true, y_pred_probs, n_classes=2)


In [ ]:
import matplotlib.pyplot as plt

# Extracting training and validation performance
epochs = [x['epoch'] for x in history]
train_perfs = [x['train_perf'] for x in history]
valid_perfs = [x['valid_perf'] for x in history]

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_perfs, label='Training Accuracy')
plt.plot(epochs, valid_perfs, label='Validation Accuracy', linestyle='--')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')  # Adjust this label if you are using a metric other than accuracy
plt.title('Accuracy vs Epoch')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

def train_and_plot(model, loader_train, loader_valid, optimizer, criterion, n_epochs, patience, device, metric=None):
    best_valid_loss = np.inf
    best_model = copy.deepcopy(model)
    waiting = 0
    history = []

    if metric is None:
        metric = balanced_accuracy_score

    for epoch in range(1, n_epochs + 1):
        train_loss, train_perf = _do_train(
            model, loader_train, optimizer, criterion, device, metric=metric)
        valid_loss, valid_perf = _validate(
            model, loader_valid, criterion, device, metric=metric)
        history.append({
            'epoch': epoch,
            'train_loss': train_loss, 'valid_loss': valid_loss,
            'train_perf': train_perf, 'valid_perf': valid_perf})

        print(f'{epoch} \t {train_loss:.4f} \t {valid_loss:.4f} \t {train_perf:.4f} \t {valid_perf:.4f}')

        # Model improvement check
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_model = copy.deepcopy(model)
            waiting = 0
        else:
            waiting += 1

        # Early stopping check
        if waiting >= patience:
            print(f'Stop training at epoch {epoch}')
            print(f'Best val loss: {best_valid_loss:.4f}')
            break

    # Accuracy vs Epoch Plot
    epochs = [x['epoch'] for x in history]
    train_accs = [x['train_perf'] for x in history]
    valid_accs = [x['valid_perf'] for x in history]

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_accs, label='Train Accuracy')
    plt.plot(epochs, valid_accs, label='Validation Accuracy', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')  # Update this if you use metrics other than accuracy
    plt.title('Accuracy vs Epoch During Training')
    plt.legend()
    plt.show()

    return best_model, history

# Update model training call
best_model, training_history = train_and_plot(model, loader_train, loader_valid, optimizer, criterion, n_epochs, patience, device, metric=cohen_kappa_score)



In [ ]:
import plotly.express as px

# Create Radar Chart
fig = px.line_polar(df, r='Precision', theta='Class', line_close=True, title='Radar Chart for Precision')
fig.add_scatterpolar(r=df['Recall'], theta=df['Class'], mode='lines', name='Recall')
fig.add_scatterpolar(r=df['F1 Score'], theta=df['Class'], mode='lines', name='F1 Score')

# Update layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True),
        angularaxis=dict(visible=True)
        
    )
)

fig.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extracted values from your classification report for 5 stages
precision = [0.77, 0.85, 0.90, 0.93, 0.80]  # Example precision values for W, N1, N2, N3, R
recall = [0.99, 0.70, 0.75, 0.80, 0.60]     # Example recall values for W, N1, N2, N3, R
f1_score = [0.86, 0.77, 0.82, 0.86, 0.68]   # Example F1 score values for W, N1, N2, N3, R
classes = ['W', 'N1', 'N2', 'N3', 'R']

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.25
index = np.arange(len(classes))

# Plot bars for each metric
bar1 = ax.bar(index, precision, bar_width, label='Precision', color='b')
bar2 = ax.bar(index + bar_width, recall, bar_width, label='Recall', color='g')
bar3 = ax.bar(index + 2 * bar_width, f1_score, bar_width, label='F1 Score', color='r')

# Add aesthetics
ax.set_xlabel('Sleep Stages')
ax.set_ylabel('Scores')
ax.set_title('Precision, Recall, and F1 Score for Each Class')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(classes)

# Change legend location to right outside the chart
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Helper function to attach a text label above each bar
def autolabel(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate('{}'.format(round(height, 2)),
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(bar1)
autolabel(bar2)
autolabel(bar3)

plt.tight_layout()
plt.show()


In [ ]:
# Plot hypnogram for one recording

mask = rec_ids == 3  # pick a recording number

t = np.arange(len(y_true[mask])) * 30 / 3600

fig, ax = plt.subplots(figsize=(12, 3))
ax.plot(t, y_true[mask], label='True')
ax.plot(t, y_pred[mask], alpha=0.7, label='Predicted')
ax.set_yticks([0, 1, 2, 3, 4])
ax.set_yticklabels(['W', 'N1', 'N2', 'N3', 'R'])
ax.set_xlabel('Time (h)')
ax.set_title('Hypnogram')
ax.legend();

In [ ]:
import matplotlib.pyplot as plt

# Extract history information
epochs = [entry['epoch'] for entry in history]
train_losses = [entry['train_loss'] for entry in history]
valid_losses = [entry['valid_loss'] for entry in history]
train_perfs = [entry['train_perf'] for entry in history]
valid_perfs = [entry['valid_perf'] for entry in history]

# Plotting
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Train')
plt.plot(epochs, valid_losses, label='Validation')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Performance plot
plt.subplot(1, 2, 2)
plt.plot(epochs, train_perfs, label='Train')
plt.plot(epochs, valid_perfs, label='Validation')
plt.title('Training and Validation Performance')
plt.xlabel('Epochs')
plt.ylabel('Performance')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Add this code after computing y_true and y_pred
import random

# Randomly select indices for visualization
sample_indices = random.sample(range(len(y_true)), min(5, len(y_true)))

# Visualize predicted vs true labels
for idx in sample_indices:
    print(f"Example {idx + 1} - True: {classes_mapping[y_true[idx]]}, Predicted: {classes_mapping[y_pred[idx]]}")
